In [51]:
import numpy as np
import json
from flask import Flask, request, jsonify, render_template
import pandas as pd
import numpy as np
import pvlib
from pvlib.modelchain import ModelChain

In [52]:
def clear_sky():
    latitude = -33.8878872
    longitude = 151.2007315
    start_date = '2020-01-01'
    end_date = '2020-01-02'
    capacity = 100

    location = pvlib.location.Location(latitude, longitude, tz='Australia/Sydney')

    times = pd.date_range(start=start_date, end=end_date, freq='1H', tz='Australia/Sydney')
    solpos = location.get_solarposition(times=times)
    dni_extra = pvlib.irradiance.get_extra_radiation(times)
    airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
    pressure = pvlib.atmosphere.alt2pres(location.altitude)
    am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
    tl = pvlib.clearsky.lookup_linke_turbidity(times, latitude, longitude)

    solis_clearsky = pvlib.clearsky.simplified_solis(solpos['apparent_zenith'], am_abs, tl)
    cs = location.get_clearsky(times, model='simplified_solis')

    capacity = float(capacity)

    system = pvlib.pvsystem.PVSystem(surface_tilt=30, surface_azimuth=180,
                                     module_parameters={'pdc0': capacity, 'gamma_pdc': -0.004},
                                     inverter_parameters={'pdc0': capacity},
                                     modules_per_string=1, strings_per_inverter=1,
                                     temperature_model_parameters={'a': -3.56, 'b': -0.075, 'deltaT': 3})
    mc = pvlib.modelchain.ModelChain(system, location, spectral_model='no_loss', aoi_model='no_loss')

    mc.run_model(solis_clearsky)

    # Replace NaN values in the 'ac' column with 0
    mc.ac.fillna(0, inplace=True)

    return mc
    

In [79]:
latitude = -33.8878872
longitude = 151.2007315
start_date = '2020-01-01'
end_date = '2020-01-02'
capacity = 100

location = pvlib.location.Location(latitude, longitude, tz='Australia/Sydney')

times = pd.date_range(start=start_date, end=end_date, freq='1H', tz='Australia/Sydney')
solpos = location.get_solarposition(times=times)
dni_extra = pvlib.irradiance.get_extra_radiation(times)
airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
pressure = pvlib.atmosphere.alt2pres(location.altitude)
am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
tl = pvlib.clearsky.lookup_linke_turbidity(times, latitude, longitude)

solis_clearsky = pvlib.clearsky.simplified_solis(solpos['apparent_zenith'], am_abs, tl)
cs = location.get_clearsky(times, model='simplified_solis')

capacity = float(capacity)

system = pvlib.pvsystem.PVSystem(surface_tilt=30, surface_azimuth=180,
                                    module_parameters={'pdc0': capacity, 'gamma_pdc': -0.004},
                                    inverter_parameters={'pdc0': capacity},
                                    modules_per_string=1, strings_per_inverter=1,
                                    temperature_model_parameters={'a': -3.56, 'b': -0.075, 'deltaT': 3})
mc = pvlib.modelchain.ModelChain(system, location, spectral_model='no_loss', aoi_model='no_loss')


In [81]:
mc.run_model(solis_clearsky.fillna(0))

ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: pvwatts_dc
  ac_model: pvwatts_inverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [85]:
mc.results.ac

2020-01-01 00:00:00+11:00     0.000000
2020-01-01 01:00:00+11:00     0.000000
2020-01-01 02:00:00+11:00     0.000000
2020-01-01 03:00:00+11:00     0.000000
2020-01-01 04:00:00+11:00     0.000000
2020-01-01 05:00:00+11:00     0.000000
2020-01-01 06:00:00+11:00     0.000000
2020-01-01 07:00:00+11:00     0.299349
2020-01-01 08:00:00+11:00     6.353778
2020-01-01 09:00:00+11:00    12.767803
2020-01-01 10:00:00+11:00    16.248710
2020-01-01 11:00:00+11:00    16.257803
2020-01-01 12:00:00+11:00    12.735190
2020-01-01 13:00:00+11:00     9.293585
2020-01-01 14:00:00+11:00    13.008783
2020-01-01 15:00:00+11:00    16.346375
2020-01-01 16:00:00+11:00    16.142269
2020-01-01 17:00:00+11:00    12.462111
2020-01-01 18:00:00+11:00     5.939298
2020-01-01 19:00:00+11:00     0.107830
2020-01-01 20:00:00+11:00     0.000000
2020-01-01 21:00:00+11:00     0.000000
2020-01-01 22:00:00+11:00     0.000000
2020-01-01 23:00:00+11:00     0.000000
2020-01-02 00:00:00+11:00     0.000000
Freq: H, Name: p_mp, dtyp

In [73]:
mc.get_ac()

AttributeError: 'ModelChain' object has no attribute 'get_ac'

In [72]:
mc.run_model(solis_clearsky)

ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: pvwatts_dc
  ac_model: pvwatts_inverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [76]:
mc

ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: pvwatts_dc
  ac_model: pvwatts_inverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [3]:
aa = clear_sky()

AttributeError: 'ModelChain' object has no attribute 'ac'

In [26]:
pd.DataFrame(aa.ac).to_dict('records')

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pvlib\modelchain.py:511: pvlibDeprecationWarning: ModelChain.ac is deprecated and will be removed in v0.10. Use ModelChain.results.ac instead
  warnings.warn(msg, pvlibDeprecationWarning)


[{'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.29934939546724537},
 {'p_mp': 6.3537777679600405},
 {'p_mp': 12.767802514482204},
 {'p_mp': 16.248709525390943},
 {'p_mp': 16.257803363775185},
 {'p_mp': 12.73519031256454},
 {'p_mp': 9.293585184661623},
 {'p_mp': 13.008782932890531},
 {'p_mp': 16.34637496673007},
 {'p_mp': 16.14226911424045},
 {'p_mp': 12.46211059124388},
 {'p_mp': 5.939298172159064},
 {'p_mp': 0.10783020005853403},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0},
 {'p_mp': 0.0}]

In [22]:
jsonify({'result':aa.ac.to_dict()})

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pvlib\modelchain.py:511: pvlibDeprecationWarning: ModelChain.ac is deprecated and will be removed in v0.10. Use ModelChain.results.ac instead
  warnings.warn(msg, pvlibDeprecationWarning)


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.